# 01. Market Expectations In The Cross-Section Of Present Values

## Imports

In [1]:
import numpy as np
import pandas as pd
import sys

from matplotlib import pyplot as plt
from settings import config
from pathlib import Path
import load_data
import regressions
import pull_CRSP_index
import pull_ken_french_data
import pull_fred

# Load environment variables
DATA_DIR = Path(config("DATA_DIR"))
OUTPUT_DIR = Path(config("OUTPUT_DIR"))
WRDS_USERNAME = config("WRDS_USERNAME")
START_DATE = config("START_DATE")
END_DATE = config("END_DATE")

## Prep Data

### Step 1: Fama-French Data

In [2]:
from pandas_datareader.famafrench import get_available_datasets
import pandas_datareader.data as web

get_available_datasets()

['F-F_Research_Data_Factors',
 'F-F_Research_Data_Factors_weekly',
 'F-F_Research_Data_Factors_daily',
 'F-F_Research_Data_5_Factors_2x3',
 'F-F_Research_Data_5_Factors_2x3_daily',
 'Portfolios_Formed_on_ME',
 'Portfolios_Formed_on_ME_Wout_Div',
 'Portfolios_Formed_on_ME_Daily',
 'Portfolios_Formed_on_BE-ME',
 'Portfolios_Formed_on_BE-ME_Wout_Div',
 'Portfolios_Formed_on_BE-ME_Daily',
 'Portfolios_Formed_on_OP',
 'Portfolios_Formed_on_OP_Wout_Div',
 'Portfolios_Formed_on_OP_Daily',
 'Portfolios_Formed_on_INV',
 'Portfolios_Formed_on_INV_Wout_Div',
 'Portfolios_Formed_on_INV_Daily',
 '6_Portfolios_2x3',
 '6_Portfolios_2x3_Wout_Div',
 '6_Portfolios_2x3_weekly',
 '6_Portfolios_2x3_daily',
 '25_Portfolios_5x5',
 '25_Portfolios_5x5_Wout_Div',
 '25_Portfolios_5x5_Daily',
 '100_Portfolios_10x10',
 '100_Portfolios_10x10_Wout_Div',
 '100_Portfolios_10x10_Daily',
 '6_Portfolios_ME_OP_2x3',
 '6_Portfolios_ME_OP_2x3_Wout_Div',
 '6_Portfolios_ME_OP_2x3_daily',
 '25_Portfolios_ME_OP_5x5',
 '25_Portf

In [3]:
bm_6_portfolios = web.DataReader("6_Portfolios_2x3", "famafrench")

/tmp/ipykernel_75939/3288387857.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  bm_6_portfolios = web.DataReader("6_Portfolios_2x3", "famafrench")
/tmp/ipykernel_75939/3288387857.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  bm_6_portfolios = web.DataReader("6_Portfolios_2x3", "famafrench")
/tmp/ipykernel_75939/3288387857.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  bm_6_portfolios = web.DataReader("6_Portfolios_2x3", "famafrench")
/tmp/ipykernel_75939/3288387857.py:1: FutureWarning: The argument 'date_parser' is

In [4]:
print(bm_6_portfolios['DESCR'])

6 Portfolios 2x3
----------------

This file was created by CMPT_ME_BEME_OP_INV_RETS using the 202412 CRSP database. It contains value- and equal-weighted returns for portfolios formed on ME and BEME. The portfolios are constructed at the end of June. BEME is book value at the last fiscal year end of the prior calendar year divided by ME at the end of December of the prior year. Annual returns are from January to December. Missing data are indicated by -99.99 or -999. The break points include utilities and include financials. The portfolios include utilities and include financials. Copyright 2024 Eugene F. Fama and Kenneth R. French

  0 : Average Value Weighted Returns -- Monthly (58 rows x 6 cols)
  1 : Average Equal Weighted Returns -- Monthly (58 rows x 6 cols)
  2 : Average Value Weighted Returns -- Annual (5 rows x 6 cols)
  3 : Average Equal Weighted Returns -- Annual (5 rows x 6 cols)
  4 : Number of Firms in Portfolios (58 rows x 6 cols)
  5 : Average Market Cap (58 rows x 6 c

In [5]:
bm_6_portfolios[6].head()

,SMALL LoBM,ME1 BM2,SMALL HiBM,BIG LoBM,ME2 BM2,BIG HiBM
Date,,,,,,
2020-03,0.1810,0.5313,1.1804,0.1574,0.5496,1.0957
2020-04,0.1813,0.5308,1.1705,0.1569,0.5448,1.0928
2020-05,0.1792,0.5297,1.1856,0.1561,0.5449,1.0953
2020-06,0.1792,0.5280,1.1877,0.1560,0.5418,1.1002
2020-07,0.1642,0.4626,1.1351,0.1252,0.4527,0.8969


### Step 2: CRSP Market Capitalization Weighted Index Data

In [6]:
# crsp = pull_CRSP_index.pull_crsp_value_weighted_index()

In [7]:
crsp_loaded = load_data.load_crsp_index()
crsp_loaded

,date,value_weighted_return
0,1930-02-01,0.058407
1,1930-03-01,0.028242
2,1930-04-01,0.074631
3,1930-05-01,-0.020541
4,1930-06-01,-0.014010
...,...,...
1123,2023-09-01,-0.020608
1124,2023-10-01,-0.047391
1125,2023-11-01,-0.028865
1126,2023-12-01,0.092053


### Step 3: Risk-Free Rate Data

In [8]:
rf = pull_fred.pull_fred()
rf

,GDP,CPIAUCNS,GDPC1,DPCREDIT,EFFR,OBFR,SOFR,DFEDTARU,DFEDTARL,WALCL,...,GFDEBTN,WTREGEN,RRPONTSYAWARD,RRPONTSYD,RPONTSYD,WSDONTL,TB3MS,Gen_IORB,ONRRP_CTPY_LIMIT,ONRP_AGG_LIMIT
DATE,,,,,,,,,,,,,,,,,,,,,
1930-01-01,NaN,17.100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1930-02-01,NaN,17.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1930-03-01,NaN,16.900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1930-04-01,NaN,17.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1930-05-01,NaN,16.900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-28,NaN,NaN,NaN,5.5,5.33,5.32,5.40,5.5,5.25,7712.781,...,NaN,731.405,5.3,829.573,0.0,38.439,NaN,5.4,160.0,500.0
2023-12-29,NaN,NaN,NaN,5.5,5.33,5.32,5.38,5.5,5.25,7712.781,...,NaN,731.405,5.3,1018.483,0.0,38.439,NaN,5.4,160.0,500.0
2023-12-30,NaN,NaN,NaN,5.5,NaN,5.32,NaN,5.5,5.25,7712.781,...,NaN,731.405,5.3,NaN,NaN,38.439,NaN,5.4,160.0,500.0


In [9]:
rf_monthly = load_data.load_fred_data()
rf_monthly

DATE
1930-01-01    0.000000
1930-02-01    0.000000
1930-03-01    0.000000
1930-04-01    0.000000
1930-05-01    0.000000
                ...   
2023-09-01    0.004433
2023-10-01    0.004450
2023-11-01    0.004392
2023-12-01    0.004367
2024-01-01    0.004350
Name: TB3MS, Length: 1129, dtype: float64

### Step 4: Excess Returns

In [10]:
ex_ret = load_data.load_and_compute_excess_returns()
ex_ret

1930-02-01    0.058407
1930-03-01    0.028242
1930-04-01    0.074631
1930-05-01   -0.020541
1930-06-01   -0.014010
                ...   
2023-09-01   -0.025041
2023-10-01   -0.051841
2023-11-01   -0.033257
2023-12-01    0.087686
2024-01-01    0.049254
Length: 1128, dtype: float64

# Run Regressions

In [11]:
import importlib
importlib.reload(load_data)

<module 'load_data' from '/home/jared/Cloud_Storage/OneDrive_jszajkowski@uchicago.edu/Courses/FINM 32900 1 (Winter 2025) Full-Stack Quantitative Finance/Project/finm32900_project_group6/src/load_data.py'>

In [12]:
# def regress():
#     portfolio_datasets = {
#         "6-Portfolios": "6_Portfolios_2x3",
#         "25-Portfolios": "25_Portfolios_5x5",
#         "100-Portfolios": "100_Portfolios_10x10"
#     }
#     results = {}
#     for label, dataset_name in portfolio_datasets.items():
#         print(f"\nProcessing {label} Data")
#         in_sample = regressions.run_in_sample_pls(dataset_name, weighting="value-weighted", h=1)
#         recursive = regressions.run_recursive_forecast(dataset_name, weighting="value-weighted", h=1)
#         regressions.display_results(label, in_sample, recursive)
#         results[label] = {"in_sample": in_sample, "recursive": recursive}
#     return results

# regress()

# Regressions for B/M ratios

In [13]:
# def regress():
#     portfolio_datasets = {
#         "6-Portfolios": "6_Portfolios_2x3",
#         # "25-Portfolios": "25_Portfolios_5x5",
#         # "100-Portfolios": "100_Portfolios_10x10"
#     }
#     results = {}
#     for label, dataset_name in portfolio_datasets.items():
#         print(f"\nProcessing {label} Data")
#         in_sample = regressions.run_in_sample_pls(
#             dataset_name=dataset_name, 
#             weighting="BE_FYt-1_to_ME_June_t", 
#             h=1, 
#             end_date='1980-01-01')
#         recursive = regressions.run_recursive_forecast(
#             dataset_name=dataset_name, 
#             weighting="BE_FYt-1_to_ME_June_t", 
#             h=1, 
#             start_train_date='1930-02-01', 
#             end_train_date='1980-01-01',
#             end_forecast_date='2011-01-01')
#         regressions.display_results(label, in_sample, recursive)
#         results[label] = {"in_sample": in_sample, "recursive": recursive}
#     return results

# regress()

# Alternate Implementation

## Step 1: Load Data


In [14]:
import pandas as pd
import numpy as np
from sklearn.cross_decomposition import PLSRegression
import statsmodels.api as sm
from sklearn.model_selection import TimeSeriesSplit

# Load Book-to-Market Ratio Data
bm_6 = load_data.load_ken_french("6_Portfolios_2x3", weighting="BE_FYt-1_to_ME_June_t").set_index("Date")
bm_25 = load_data.load_ken_french("25_Portfolios_5x5", weighting="BE_FYt-1_to_ME_June_t").set_index("Date")
bm_100 = load_data.load_ken_french("100_Portfolios_10x10", weighting="BE_FYt-1_to_ME_June_t").set_index("Date")
bm_6

,SMALL LoBM,ME1 BM2,SMALL HiBM,BIG LoBM,ME2 BM2,BIG HiBM
Date,,,,,,
1930-01-01,0.3712,0.8187,2.4928,0.2347,0.6657,1.2921
1930-02-01,0.3703,0.8226,2.5153,0.2340,0.6652,1.2914
1930-03-01,0.3726,0.8214,2.5149,0.2332,0.6596,1.2903
1930-04-01,0.3686,0.8272,2.5144,0.2326,0.6611,1.2897
1930-05-01,0.3662,0.8265,2.4851,0.2316,0.6610,1.2899
...,...,...,...,...,...,...
2023-09-01,0.1653,0.4784,1.2106,0.1068,0.4587,0.9884
2023-10-01,0.1658,0.4810,1.2139,0.1075,0.4607,0.9874
2023-11-01,0.1650,0.4827,1.2040,0.1069,0.4612,0.9852


In [15]:
# Load Market Returns and Compute Excess Returns
market_returns = load_data.load_crsp_index()
market_returns.rename(columns={"date": "Date", "value_weighted_return": "Market Return"}, inplace=True)
market_returns.set_index("Date", inplace=True)
market_returns

,Market Return
Date,
1930-02-01,0.058407
1930-03-01,0.028242
1930-04-01,0.074631
1930-05-01,-0.020541
1930-06-01,-0.014010
...,...
2023-09-01,-0.020608
2023-10-01,-0.047391
2023-11-01,-0.028865


In [16]:
rf_monthly = load_data.load_fred_data()
rf_monthly

DATE
1930-01-01    0.000000
1930-02-01    0.000000
1930-03-01    0.000000
1930-04-01    0.000000
1930-05-01    0.000000
                ...   
2023-09-01    0.004433
2023-10-01    0.004450
2023-11-01    0.004392
2023-12-01    0.004367
2024-01-01    0.004350
Name: TB3MS, Length: 1129, dtype: float64

In [17]:
market_returns["Risk-Free Rate"] = rf_monthly
market_returns

,Market Return,Risk-Free Rate
Date,,
1930-02-01,0.058407,0.000000
1930-03-01,0.028242,0.000000
1930-04-01,0.074631,0.000000
1930-05-01,-0.020541,0.000000
1930-06-01,-0.014010,0.000000
...,...,...
2023-09-01,-0.020608,0.004433
2023-10-01,-0.047391,0.004450
2023-11-01,-0.028865,0.004392


In [18]:
market_returns["Excess Return"] = market_returns["Market Return"] - market_returns["Risk-Free Rate"]
market_returns

,Market Return,Risk-Free Rate,Excess Return
Date,,,
1930-02-01,0.058407,0.000000,0.058407
1930-03-01,0.028242,0.000000,0.028242
1930-04-01,0.074631,0.000000,0.074631
1930-05-01,-0.020541,0.000000,-0.020541
1930-06-01,-0.014010,0.000000,-0.014010
...,...,...,...
2023-09-01,-0.020608,0.004433,-0.025041
2023-10-01,-0.047391,0.004450,-0.051841
2023-11-01,-0.028865,0.004392,-0.033257


In [19]:
market_returns_in_sample = market_returns[market_returns.index <= "1980-01-01"]
market_returns_in_sample

,Market Return,Risk-Free Rate,Excess Return
Date,,,
1930-02-01,0.058407,0.000000,0.058407
1930-03-01,0.028242,0.000000,0.028242
1930-04-01,0.074631,0.000000,0.074631
1930-05-01,-0.020541,0.000000,-0.020541
1930-06-01,-0.014010,0.000000,-0.014010
...,...,...,...
1979-09-01,0.064732,0.008550,0.056182
1979-10-01,0.001426,0.009750,-0.008324
1979-11-01,-0.072662,0.009825,-0.082487


In [20]:
# Ensure all data is aligned on the Date index
bm_6 = bm_6.loc[market_returns.index]
bm_25 = bm_25.loc[market_returns.index]
bm_100 = bm_100.loc[market_returns.index]

# Select different portfolio sizes for processing
bm_dict = {"6 Portfolios": bm_6, "25 Portfolios": bm_25, "100 Portfolios": bm_100}
bm_dict

{'6 Portfolios':             SMALL LoBM  ME1 BM2  SMALL HiBM  BIG LoBM  ME2 BM2  BIG HiBM
 Date                                                                    
 1930-02-01      0.3703   0.8226      2.5153    0.2340   0.6652    1.2914
 1930-03-01      0.3726   0.8214      2.5149    0.2332   0.6596    1.2903
 1930-04-01      0.3686   0.8272      2.5144    0.2326   0.6611    1.2897
 1930-05-01      0.3662   0.8265      2.4851    0.2316   0.6610    1.2899
 1930-06-01      0.3647   0.8240      2.4542    0.2327   0.6610    1.2865
 ...                ...      ...         ...       ...      ...       ...
 2023-09-01      0.1653   0.4784      1.2106    0.1068   0.4587    0.9884
 2023-10-01      0.1658   0.4810      1.2139    0.1075   0.4607    0.9874
 2023-11-01      0.1650   0.4827      1.2040    0.1069   0.4612    0.9852
 2023-12-01      0.1638   0.4823      1.1833    0.1052   0.4590    0.9860
 2024-01-01      0.1642   0.4828      1.1624    0.1057   0.4576    0.9873
 
 [1128 rows x 6 colu

In [21]:
# Ensure all data is aligned on the Date index
bm_6_in_sample = bm_6.loc[market_returns_in_sample.index]
bm_25_in_sample = bm_25.loc[market_returns_in_sample.index]
bm_100_in_sample = bm_100.loc[market_returns_in_sample.index]

# Select different portfolio sizes for processing
bm_dict_in_sample = {"6 Portfolios In Sample": bm_6_in_sample, 
                     "25 Portfolios In Sample": bm_25_in_sample, 
                     "100 Portfolios In Sample": bm_100_in_sample}
bm_dict_in_sample

{'6 Portfolios In Sample':             SMALL LoBM  ME1 BM2  SMALL HiBM  BIG LoBM  ME2 BM2  BIG HiBM
 Date                                                                    
 1930-02-01      0.3703   0.8226      2.5153    0.2340   0.6652    1.2914
 1930-03-01      0.3726   0.8214      2.5149    0.2332   0.6596    1.2903
 1930-04-01      0.3686   0.8272      2.5144    0.2326   0.6611    1.2897
 1930-05-01      0.3662   0.8265      2.4851    0.2316   0.6610    1.2899
 1930-06-01      0.3647   0.8240      2.4542    0.2327   0.6610    1.2865
 ...                ...      ...         ...       ...      ...       ...
 1979-09-01      0.4655   1.0421      1.7298    0.5332   1.0922    1.7187
 1979-10-01      0.4622   1.0405      1.7218    0.5343   1.0891    1.7156
 1979-11-01      0.4630   1.0386      1.7197    0.5332   1.0879    1.7103
 1979-12-01      0.4551   1.0350      1.7146    0.5328   1.0849    1.6953
 1980-01-01      0.4491   1.0302      1.7091    0.5334   1.0833    1.7003
 
 [600 rows

## Step 2: Implement the PLS Regression

In [22]:
def compute_pls_predictor(X, y, n_components=1):
    """
    Compute the PLS predictor factor from a cross-section of book-to-market ratios.
    X: Book-to-market ratios (DataFrame)
    y: Excess market returns (Series)
    """
    pls = PLSRegression(n_components=n_components)
    pls.fit(X, y)
    factor = pls.transform(X)  # Extract the PLS factor
    return factor.flatten()  # Convert to 1D array

def predictive_regression(factor, excess_returns):
    """
    Run a predictive regression: Excess market returns ~ Extracted PLS Factor
    """
    X = sm.add_constant(factor)
    model = sm.OLS(excess_returns, X).fit()
    return model

# Store results
results = {}

for name, bm_data in bm_dict_in_sample.items():
    print(f"\nRunning PLS + Regression for {name}")

    # Extract PLS factor
    pls_factor = compute_pls_predictor(bm_data, market_returns_in_sample["Excess Return"])
    # pls_df = pd.DataFrame(pls_factor, columns=["PLS Factor"])
    # print(pls_df)

    # Run predictive regression
    model = predictive_regression(pls_factor[:-1], market_returns_in_sample["Excess Return"].iloc[1:])  # Shift for forecasting

    # Store results
    results[name] = {
        "R2 In-Sample": model.rsquared,
        "Model Summary": model.summary()
    }

    print(f"R2 In-Sample for {name}: {model.rsquared:.4f}")



Running PLS + Regression for 6 Portfolios In Sample
R2 In-Sample for 6 Portfolios In Sample: 0.0418

Running PLS + Regression for 25 Portfolios In Sample
R2 In-Sample for 25 Portfolios In Sample: 0.0415

Running PLS + Regression for 100 Portfolios In Sample
R2 In-Sample for 100 Portfolios In Sample: 0.0490


## Step 3: Compute Out-of-Sample Predictions (Recursive Forecast)


In [23]:
import numpy as np
import statsmodels.api as sm

def out_of_sample_forecast(X, y, split_date):
    """
    Perform a rolling out-of-sample forecast using PLS regression.
    X: Book-to-market ratios (features)
    y: Excess market returns (target)
    split_date: Date at which training ends and forecasting begins
    """
    train_mask = X.index < split_date
    test_mask = X.index >= split_date

    X_train, X_test = X[train_mask], X[test_mask]
    y_train, y_test = y[train_mask], y[test_mask]
    print(y_test)

    # Train PLS on pre-1980 data
    pls_factor_train = compute_pls_predictor(X_train, y_train)

    # Ensure y_train matches the shape of extracted factors
    y_train = y_train.iloc[:len(pls_factor_train)]  

    # Initial predictive regression
    X_train_const = sm.add_constant(pls_factor_train)
    model = sm.OLS(y_train, X_train_const).fit()

    # Rolling Forecast
    predictions = []
    actuals = []

    for i in range(len(X_test)):
        # Expanding window approach
        X_train_curr = X.iloc[:len(y_train) + i]
        y_train_curr = y.iloc[:len(y_train) + i]
        print(f"Shape of y_train_curr: {y_train_curr.shape}")

        # Extract PLS factor for new training window
        pls_factor = compute_pls_predictor(X_train_curr, y_train_curr)
        print(f"Shape of PLS Factor: {pls_factor.shape}")
        # print(pls_factor)

        # Ensure sizes match
        y_train_curr = y_train_curr.iloc[:len(pls_factor)]
        print(f"Shape of y_train_curr: {y_train_curr.shape}")

        # Fit predictive regression with expanding window
        X_train_curr = sm.add_constant(pls_factor)
        model = sm.OLS(y_train_curr, X_train_curr).fit()
        print(f"Shape of X_train_curr: {X_train_curr.shape}")
        # print(f"X_train_curr: {X_train_curr}")

        # **Final Fix: Ensure Shape (1,2)**
        # X_curr = sm.add_constant(np.array([[pls_factor[-1]]]))  # Fix: no indexing, just reshape
        X_curr = sm.add_constant(np.array([pls_factor])[-1])  # Fix: no indexing, just reshape
        print(f"Shape of X_curr: {X_curr.shape}")
        # print(f"X_curr: {X_curr}")



        print(f"\nIteration {i+1}")
        print(f"Final Fixed Shape of X_curr: {X_curr.shape}")  # Should be (1,2)
        print(f"Shape of model.params: {model.params.shape}")  # Should be (2,)

        # Prediction
        prediction = model.predict(X_curr)[0]  # Ensure single value is returned
        # prediction = model.predict(X_curr)  # Ensure single value is returned
        print(f"Prediction: {prediction}")
        predictions.append(prediction)
        print(f"y test {y_test.iloc[i]}")
        actuals.append(y_test.iloc[i])

    # Convert lists to numpy arrays
    print(len(actuals))
    print(actuals)
    actuals = np.array(actuals)
    
    print(len(predictions))
    print(predictions)
    # predictions = np.array(predictions)
    

    # Compute Out-of-Sample R²
    r2_out_sample = 1 - np.sum((actuals - predictions) ** 2) / np.sum((actuals - np.mean(actuals)) ** 2)

    return r2_out_sample


split_date = "1980-01-01"

for name, bm_data in bm_dict.items():
    print(f"\nRunning Out-of-Sample Forecast for {name}")

    # Compute Out-of-Sample R²
    r2_out_sample = out_of_sample_forecast(bm_data, market_returns["Excess Return"], split_date)

    # Ensure there's an entry for this dataset
    if name not in results:
        results[name] = {}

    # Store Out-of-Sample R²
    results[name]["R2 Out-of-Sample"] = r2_out_sample

    print(f"R2 Out-of-Sample for {name}: {r2_out_sample:.4f}")



Running Out-of-Sample Forecast for 6 Portfolios
Date
1980-01-01    0.018215
1980-02-01    0.054890
1980-03-01   -0.011652
1980-04-01   -0.131224
1980-05-01    0.045137
                ...   
2023-09-01   -0.025041
2023-10-01   -0.051841
2023-11-01   -0.033257
2023-12-01    0.087686
2024-01-01    0.049254
Name: Excess Return, Length: 529, dtype: float64
Shape of y_train_curr: (599,)
Shape of PLS Factor: (599,)
Shape of y_train_curr: (599,)
Shape of X_train_curr: (599, 2)
Shape of X_curr: (599, 2)

Iteration 1
Final Fixed Shape of X_curr: (599, 2)
Shape of model.params: (2,)
Prediction: -0.00021562451997542316
y test 0.018215000000000002
Shape of y_train_curr: (600,)
Shape of PLS Factor: (600,)
Shape of y_train_curr: (600,)
Shape of X_train_curr: (600, 2)
Shape of X_curr: (600, 2)

Iteration 2
Final Fixed Shape of X_curr: (600, 2)
Shape of model.params: (2,)
Prediction: -0.00019121830096807506
y test 0.05489033333333333
Shape of y_train_curr: (601,)
Shape of PLS Factor: (601,)
Shape of 

## Step 4: Report Results (Comparison to Table 1)


In [24]:
import pandas as pd


# Create a DataFrame to summarize results
summary_df = pd.DataFrame(results).T[['R2 In-Sample', 'R2 Out-of-Sample']]
summary_df.columns = ["In-Sample R2", "Out-of-Sample R2"]

# Display the results
summary_df

,In-Sample R2,Out-of-Sample R2
6 Portfolios In Sample,0.041799,NaN
25 Portfolios In Sample,0.04147,NaN
100 Portfolios In Sample,0.049015,NaN
6 Portfolios,NaN,-0.011686
25 Portfolios,NaN,-0.010864
100 Portfolios,NaN,-0.012236


# Alternate Implementation - Annual

## Step 1: Load Data


In [25]:
import pandas as pd
import numpy as np
from sklearn.cross_decomposition import PLSRegression
import statsmodels.api as sm
from sklearn.model_selection import TimeSeriesSplit

# Load Book-to-Market Ratio Data
bm_6 = load_data.load_ken_french("6_Portfolios_2x3", weighting="BE_FYt-1_to_ME_June_t").set_index("Date")
bm_25 = load_data.load_ken_french("25_Portfolios_5x5", weighting="BE_FYt-1_to_ME_June_t").set_index("Date")
bm_100 = load_data.load_ken_french("100_Portfolios_10x10", weighting="BE_FYt-1_to_ME_June_t").set_index("Date")
bm_6

,SMALL LoBM,ME1 BM2,SMALL HiBM,BIG LoBM,ME2 BM2,BIG HiBM
Date,,,,,,
1930-01-01,0.3712,0.8187,2.4928,0.2347,0.6657,1.2921
1930-02-01,0.3703,0.8226,2.5153,0.2340,0.6652,1.2914
1930-03-01,0.3726,0.8214,2.5149,0.2332,0.6596,1.2903
1930-04-01,0.3686,0.8272,2.5144,0.2326,0.6611,1.2897
1930-05-01,0.3662,0.8265,2.4851,0.2316,0.6610,1.2899
...,...,...,...,...,...,...
2023-09-01,0.1653,0.4784,1.2106,0.1068,0.4587,0.9884
2023-10-01,0.1658,0.4810,1.2139,0.1075,0.4607,0.9874
2023-11-01,0.1650,0.4827,1.2040,0.1069,0.4612,0.9852


In [26]:
# Load Market Returns and Compute Excess Returns
market_returns = load_data.load_crsp_index()
market_returns.rename(columns={"date": "Date", "value_weighted_return": "Market Return"}, inplace=True)
market_returns.set_index("Date", inplace=True)
market_returns

,Market Return
Date,
1930-02-01,0.058407
1930-03-01,0.028242
1930-04-01,0.074631
1930-05-01,-0.020541
1930-06-01,-0.014010
...,...
2023-09-01,-0.020608
2023-10-01,-0.047391
2023-11-01,-0.028865


In [27]:
rf_monthly = load_data.load_fred_data()
rf_monthly

DATE
1930-01-01    0.000000
1930-02-01    0.000000
1930-03-01    0.000000
1930-04-01    0.000000
1930-05-01    0.000000
                ...   
2023-09-01    0.004433
2023-10-01    0.004450
2023-11-01    0.004392
2023-12-01    0.004367
2024-01-01    0.004350
Name: TB3MS, Length: 1129, dtype: float64

In [28]:
market_returns["Risk-Free Rate"] = rf_monthly
market_returns

,Market Return,Risk-Free Rate
Date,,
1930-02-01,0.058407,0.000000
1930-03-01,0.028242,0.000000
1930-04-01,0.074631,0.000000
1930-05-01,-0.020541,0.000000
1930-06-01,-0.014010,0.000000
...,...,...
2023-09-01,-0.020608,0.004433
2023-10-01,-0.047391,0.004450
2023-11-01,-0.028865,0.004392


In [29]:
market_returns["Excess Return"] = market_returns["Market Return"] - market_returns["Risk-Free Rate"]
market_returns

,Market Return,Risk-Free Rate,Excess Return
Date,,,
1930-02-01,0.058407,0.000000,0.058407
1930-03-01,0.028242,0.000000,0.028242
1930-04-01,0.074631,0.000000,0.074631
1930-05-01,-0.020541,0.000000,-0.020541
1930-06-01,-0.014010,0.000000,-0.014010
...,...,...,...
2023-09-01,-0.020608,0.004433,-0.025041
2023-10-01,-0.047391,0.004450,-0.051841
2023-11-01,-0.028865,0.004392,-0.033257


In [30]:
market_returns_in_sample = market_returns[market_returns.index <= "1980-01-01"]
market_returns_in_sample

,Market Return,Risk-Free Rate,Excess Return
Date,,,
1930-02-01,0.058407,0.000000,0.058407
1930-03-01,0.028242,0.000000,0.028242
1930-04-01,0.074631,0.000000,0.074631
1930-05-01,-0.020541,0.000000,-0.020541
1930-06-01,-0.014010,0.000000,-0.014010
...,...,...,...
1979-09-01,0.064732,0.008550,0.056182
1979-10-01,0.001426,0.009750,-0.008324
1979-11-01,-0.072662,0.009825,-0.082487


In [31]:
# Ensure all data is aligned on the Date index
bm_6 = bm_6.loc[market_returns.index]
bm_25 = bm_25.loc[market_returns.index]
bm_100 = bm_100.loc[market_returns.index]

# Select different portfolio sizes for processing
bm_dict = {"6 Portfolios": bm_6, "25 Portfolios": bm_25, "100 Portfolios": bm_100}
bm_dict

{'6 Portfolios':             SMALL LoBM  ME1 BM2  SMALL HiBM  BIG LoBM  ME2 BM2  BIG HiBM
 Date                                                                    
 1930-02-01      0.3703   0.8226      2.5153    0.2340   0.6652    1.2914
 1930-03-01      0.3726   0.8214      2.5149    0.2332   0.6596    1.2903
 1930-04-01      0.3686   0.8272      2.5144    0.2326   0.6611    1.2897
 1930-05-01      0.3662   0.8265      2.4851    0.2316   0.6610    1.2899
 1930-06-01      0.3647   0.8240      2.4542    0.2327   0.6610    1.2865
 ...                ...      ...         ...       ...      ...       ...
 2023-09-01      0.1653   0.4784      1.2106    0.1068   0.4587    0.9884
 2023-10-01      0.1658   0.4810      1.2139    0.1075   0.4607    0.9874
 2023-11-01      0.1650   0.4827      1.2040    0.1069   0.4612    0.9852
 2023-12-01      0.1638   0.4823      1.1833    0.1052   0.4590    0.9860
 2024-01-01      0.1642   0.4828      1.1624    0.1057   0.4576    0.9873
 
 [1128 rows x 6 colu

### Convert to annual ratios - Full Sample

In [32]:
# Resample: Take the last available observation in each year
bm_6_annual = bm_6.resample('YE').last()
bm_25_annual = bm_25.resample('YE').last()
bm_100_annual = bm_100.resample('YE').last()

bm_dict_annual = {"6 Portfolios - Annual": bm_6_annual, 
                  "25 Portfolios - Annual": bm_25_annual, 
                  "100 Portfolios - Annual": bm_100_annual}
bm_dict_annual

{'6 Portfolios - Annual':             SMALL LoBM  ME1 BM2  SMALL HiBM  BIG LoBM  ME2 BM2  BIG HiBM
 Date                                                                    
 1930-12-31      0.6251   1.4497      4.1054    0.4379   1.0711    2.2919
 1931-12-31      0.8313   2.0927      6.6355    0.6293   1.7446    4.1302
 1932-12-31      2.7729   7.3239     31.3614    1.7544   6.8427   19.6392
 1933-12-31      0.7365   1.5291      7.0439    0.7103   1.6389    3.8141
 1934-12-31      0.6860   1.5848      6.8630    0.5660   1.3978    4.7152
 ...                ...      ...         ...       ...      ...       ...
 2020-12-31      0.1994   0.5932      1.6704    0.1174   0.5506    1.2556
 2021-12-31      0.1342   0.3641      0.9081    0.0846   0.3608    0.7530
 2022-12-31      0.2278   0.5170      1.1896    0.1185   0.4410    0.8404
 2023-12-31      0.1638   0.4823      1.1833    0.1052   0.4590    0.9860
 2024-12-31      0.1642   0.4828      1.1624    0.1057   0.4576    0.9873
 
 [95 rows x

In [33]:
# Ensure all data is aligned on the Date index
bm_6_in_sample = bm_6.loc[market_returns_in_sample.index]
bm_25_in_sample = bm_25.loc[market_returns_in_sample.index]
bm_100_in_sample = bm_100.loc[market_returns_in_sample.index]

# Select different portfolio sizes for processing
bm_dict_in_sample = {"6 Portfolios In Sample": bm_6_in_sample, 
                     "25 Portfolios In Sample": bm_25_in_sample, 
                     "100 Portfolios In Sample": bm_100_in_sample}
bm_dict_in_sample

{'6 Portfolios In Sample':             SMALL LoBM  ME1 BM2  SMALL HiBM  BIG LoBM  ME2 BM2  BIG HiBM
 Date                                                                    
 1930-02-01      0.3703   0.8226      2.5153    0.2340   0.6652    1.2914
 1930-03-01      0.3726   0.8214      2.5149    0.2332   0.6596    1.2903
 1930-04-01      0.3686   0.8272      2.5144    0.2326   0.6611    1.2897
 1930-05-01      0.3662   0.8265      2.4851    0.2316   0.6610    1.2899
 1930-06-01      0.3647   0.8240      2.4542    0.2327   0.6610    1.2865
 ...                ...      ...         ...       ...      ...       ...
 1979-09-01      0.4655   1.0421      1.7298    0.5332   1.0922    1.7187
 1979-10-01      0.4622   1.0405      1.7218    0.5343   1.0891    1.7156
 1979-11-01      0.4630   1.0386      1.7197    0.5332   1.0879    1.7103
 1979-12-01      0.4551   1.0350      1.7146    0.5328   1.0849    1.6953
 1980-01-01      0.4491   1.0302      1.7091    0.5334   1.0833    1.7003
 
 [600 rows

### Convert to annual returns - Full sample

In [34]:
# Load your data (assuming it is in a DataFrame named market_returns)
# market_returns.index = pd.to_datetime(market_returns.index)  # Ensure Date is in datetime format

# Annual Market Return (Cumulative Product for Total Return)
market_returns['Market Return'] = market_returns['Market Return'] + 1  # Convert to growth factor
market_returns

,Market Return,Risk-Free Rate,Excess Return
Date,,,
1930-02-01,1.058407,0.000000,0.058407
1930-03-01,1.028242,0.000000,0.028242
1930-04-01,1.074631,0.000000,0.074631
1930-05-01,0.979459,0.000000,-0.020541
1930-06-01,0.985990,0.000000,-0.014010
...,...,...,...
2023-09-01,0.979392,0.004433,-0.025041
2023-10-01,0.952609,0.004450,-0.051841
2023-11-01,0.971135,0.004392,-0.033257


In [35]:
annual_market_return = market_returns['Market Return'].resample('YE').prod() - 1  # Convert back to returns

# Annual Risk-Free Rate (Sum of Monthly Rates)
annual_risk_free_rate = market_returns['Risk-Free Rate'].resample('YE').sum()

# Compute Annual Excess Return
annual_excess_return = annual_market_return - annual_risk_free_rate

# Create New Annual DataFrame
annual_returns = pd.DataFrame({
    "Market Return": annual_market_return,
    "Risk-Free Rate": annual_risk_free_rate,
    "Excess Return": annual_excess_return
})

# Adjust index to represent the year (instead of Dec 31st)
# annual_returns.index = annual_returns.index.year

# Display the transformed annual data
annual_returns

,Market Return,Risk-Free Rate,Excess Return
Date,,,
1930-12-31,-0.229073,0.000000,-0.229073
1931-12-31,-0.407084,0.000000,-0.407084
1932-12-31,-0.238152,0.000000,-0.238152
1933-12-31,0.616427,0.000000,0.616427
1934-12-31,0.046574,0.002783,0.043791
...,...,...,...
2020-12-31,0.188234,0.003650,0.184584
2021-12-31,0.244975,0.000442,0.244533
2022-12-31,-0.117508,0.020217,-0.137725


In [36]:
annual_returns_in_sample = annual_returns[annual_returns.index <= "1980-01-01"]
annual_returns_in_sample

,Market Return,Risk-Free Rate,Excess Return
Date,,,
1930-12-31,-0.229073,0.000000,-0.229073
1931-12-31,-0.407084,0.000000,-0.407084
1932-12-31,-0.238152,0.000000,-0.238152
1933-12-31,0.616427,0.000000,0.616427
1934-12-31,0.046574,0.002783,0.043791
1935-12-31,0.394229,0.001675,0.392554
1936-12-31,0.381380,0.001725,0.379655
1937-12-31,-0.316339,0.002758,-0.319097
1938-12-31,0.182425,0.000650,0.181775


### Convert to annual ratios - In Sample

In [37]:
# Ensure all data is aligned on the Date index
bm_6_in_sample_annual = bm_6_annual.loc[annual_returns_in_sample.index]
bm_25_in_sample_annual = bm_25_annual.loc[annual_returns_in_sample.index]
bm_100_in_sample_annual = bm_100_annual.loc[annual_returns_in_sample.index]

# Select different portfolio sizes for processing
bm_dict_in_sample_annual = {"6 Portfolios - In Sample Annual": bm_6_in_sample_annual, 
                            "25 Portfolios - In Sampole Annual": bm_25_in_sample_annual, 
                            "100 Portfolios - In Sample Annual": bm_100_in_sample_annual}
bm_dict_in_sample_annual

{'6 Portfolios - In Sample Annual':             SMALL LoBM  ME1 BM2  SMALL HiBM  BIG LoBM  ME2 BM2  BIG HiBM
 Date                                                                    
 1930-12-31      0.6251   1.4497      4.1054    0.4379   1.0711    2.2919
 1931-12-31      0.8313   2.0927      6.6355    0.6293   1.7446    4.1302
 1932-12-31      2.7729   7.3239     31.3614    1.7544   6.8427   19.6392
 1933-12-31      0.7365   1.5291      7.0439    0.7103   1.6389    3.8141
 1934-12-31      0.6860   1.5848      6.8630    0.5660   1.3978    4.7152
 1935-12-31      0.7315   1.8820     11.4219    0.4575   1.2944    5.5439
 1936-12-31      0.4693   1.0483      5.2232    0.3015   0.8879    2.5646
 1937-12-31      0.3914   0.8795      4.6186    0.3239   0.7848    1.8426
 1938-12-31      0.6243   1.3443      6.4539    0.5325   1.2311    4.2579
 1939-12-31      0.6607   1.5817      8.3881    0.4596   1.2525    4.9726
 1940-12-31      0.6993   1.5884      6.2476    0.5328   1.4219    4.5116
 19

## Step 2: Implement the PLS Regression - Annual

In [38]:
def compute_pls_predictor(X, y, n_components=1):
    """
    Compute the PLS predictor factor from a cross-section of book-to-market ratios.
    X: Book-to-market ratios (DataFrame)
    y: Excess market returns (Series)
    """
    pls = PLSRegression(n_components=n_components)
    pls.fit(X, y)
    factor = pls.transform(X)  # Extract the PLS factor
    return factor.flatten()  # Convert to 1D array

def predictive_regression(factor, excess_returns):
    """
    Run a predictive regression: Excess market returns ~ Extracted PLS Factor
    """
    X = sm.add_constant(factor)
    model = sm.OLS(excess_returns, X).fit()
    return model

# Store results
results = {}

for name, bm_data in bm_dict_in_sample_annual.items():
    print(f"\nRunning PLS + Regression for {name}")

    # Extract PLS factor
    pls_factor = compute_pls_predictor(bm_data, annual_returns_in_sample["Excess Return"])
    # pls_df = pd.DataFrame(pls_factor, columns=["PLS Factor"])
    # print(pls_df)

    # Run predictive regression
    model = predictive_regression(pls_factor[:-1], annual_returns_in_sample["Excess Return"].iloc[1:])  # Shift for forecasting

    # Store results
    results[name] = {
        "R2 In-Sample": model.rsquared,
        "Model Summary": model.summary()
    }

    print(f"R2 In-Sample for {name}: {model.rsquared:.4f}")
    print(f"Model summary for {name}: ", model.summary())



Running PLS + Regression for 6 Portfolios - In Sample Annual
R2 In-Sample for 6 Portfolios - In Sample Annual: 0.1674
Model summary for 6 Portfolios - In Sample Annual:                              OLS Regression Results                            
Dep. Variable:          Excess Return   R-squared:                       0.167
Model:                            OLS   Adj. R-squared:                  0.150
Method:                 Least Squares   F-statistic:                     9.453
Date:                Wed, 12 Mar 2025   Prob (F-statistic):            0.00351
Time:                        00:51:36   Log-Likelihood:                 10.661
No. Observations:                  49   AIC:                            -17.32
Df Residuals:                      47   BIC:                            -13.54
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err       

## Step 3: Compute Out-of-Sample Predictions (Recursive Forecast)


In [39]:
import numpy as np
import statsmodels.api as sm

def out_of_sample_forecast(X, y, split_date):
    """
    Perform a rolling out-of-sample forecast using PLS regression.
    X: Book-to-market ratios (features)
    y: Excess market returns (target)
    split_date: Date at which training ends and forecasting begins
    """
    train_mask = X.index < split_date
    test_mask = X.index >= split_date

    X_train, X_test = X[train_mask], X[test_mask]
    y_train, y_test = y[train_mask], y[test_mask]
    print(y_test)

    # Train PLS on pre-1980 data
    pls_factor_train = compute_pls_predictor(X_train, y_train)

    # Ensure y_train matches the shape of extracted factors
    y_train = y_train.iloc[:len(pls_factor_train)]  

    # Initial predictive regression
    X_train_const = sm.add_constant(pls_factor_train)
    model = sm.OLS(y_train, X_train_const).fit()

    # Rolling Forecast
    predictions = []
    actuals = []

    for i in range(len(X_test)):
        # Expanding window approach
        X_train_curr = X.iloc[:len(y_train) + i]
        y_train_curr = y.iloc[:len(y_train) + i]
        print(f"Shape of y_train_curr: {y_train_curr.shape}")

        # Extract PLS factor for new training window
        pls_factor = compute_pls_predictor(X_train_curr, y_train_curr)
        print(f"Shape of PLS Factor: {pls_factor.shape}")
        # print(pls_factor)

        # Ensure sizes match
        y_train_curr = y_train_curr.iloc[:len(pls_factor)]
        print(f"Shape of y_train_curr: {y_train_curr.shape}")

        # Fit predictive regression with expanding window
        X_train_curr = sm.add_constant(pls_factor)
        model = sm.OLS(y_train_curr, X_train_curr).fit()
        print(f"Shape of X_train_curr: {X_train_curr.shape}")
        # print(f"X_train_curr: {X_train_curr}")

        # **Final Fix: Ensure Shape (1,2)**
        # X_curr = sm.add_constant(np.array([[pls_factor[-1]]]))  # Fix: no indexing, just reshape
        X_curr = sm.add_constant(np.array([pls_factor])[-1])  # Fix: no indexing, just reshape
        print(f"Shape of X_curr: {X_curr.shape}")
        # print(f"X_curr: {X_curr}")



        print(f"\nIteration {i+1}")
        print(f"Final Fixed Shape of X_curr: {X_curr.shape}")  # Should be (1,2)
        print(f"Shape of model.params: {model.params.shape}")  # Should be (2,)

        # Prediction
        prediction = model.predict(X_curr)[0]  # Ensure single value is returned
        # prediction = model.predict(X_curr)  # Ensure single value is returned
        print(f"Prediction: {prediction}")
        predictions.append(prediction)
        print(f"y test {y_test.iloc[i]}")
        actuals.append(y_test.iloc[i])

    # Convert lists to numpy arrays
    print(len(actuals))
    print(actuals)
    actuals = np.array(actuals)
    
    print(len(predictions))
    print(predictions)
    # predictions = np.array(predictions)
    

    # Compute Out-of-Sample R²
    r2_out_sample = 1 - np.sum((actuals - predictions) ** 2) / np.sum((actuals - np.mean(actuals)) ** 2)

    return r2_out_sample


split_date = "1980-01-01"

for name, bm_data in bm_dict_annual.items():
    print(f"\nRunning Out-of-Sample Forecast for {name}")

    # Compute Out-of-Sample R²
    r2_out_sample = out_of_sample_forecast(bm_data, annual_returns["Excess Return"], split_date)

    # Ensure there's an entry for this dataset
    if name not in results:
        results[name] = {}

    # Store Out-of-Sample R²
    results[name]["R2 Out-of-Sample"] = r2_out_sample

    print(f"R2 Out-of-Sample for {name}: {r2_out_sample:.4f}")



Running Out-of-Sample Forecast for 6 Portfolios - Annual
Date
1980-12-31    0.304470
1981-12-31   -0.186364
1982-12-31    0.047506
1983-12-31    0.171314
1984-12-31   -0.097965
1985-12-31    0.214871
1986-12-31    0.178278
1987-12-31   -0.131448
1988-12-31    0.165580
1989-12-31    0.206716
1990-12-31   -0.145984
1991-12-31    0.189403
1992-12-31    0.151732
1993-12-31    0.083901
1994-12-31   -0.043092
1995-12-31    0.298234
1996-12-31    0.194237
1997-12-31    0.215439
1998-12-31    0.122661
1999-12-31    0.182426
2000-12-31   -0.114659
2001-12-31   -0.144315
2002-12-31   -0.164832
2003-12-31    0.195475
2004-12-31    0.127444
2005-12-31    0.075486
2006-12-31    0.106447
2007-12-31    0.045589
2008-12-31   -0.412148
2009-12-31    0.303809
2010-12-31    0.132991
2011-12-31    0.050985
2012-12-31    0.146758
2013-12-31    0.286717
2014-12-31    0.137577
2015-12-31    0.001395
2016-12-31    0.078083
2017-12-31    0.204968
2018-12-31    0.022536
2019-12-31    0.123651
2020-12-31    0.1

## Step 4: Report Results (Comparison to Table 1)


In [40]:
import pandas as pd


# Create a DataFrame to summarize results
summary_df = pd.DataFrame(results).T[['R2 In-Sample', 'R2 Out-of-Sample']]
summary_df.columns = ["In-Sample R2", "Out-of-Sample R2"]

# Display the results
summary_df

,In-Sample R2,Out-of-Sample R2
6 Portfolios - In Sample Annual,0.167443,NaN
25 Portfolios - In Sampole Annual,0.155118,NaN
100 Portfolios - In Sample Annual,0.114806,NaN
6 Portfolios - Annual,NaN,-0.019181
25 Portfolios - Annual,NaN,-0.027233
100 Portfolios - Annual,NaN,-0.135839
